In [18]:
import pandas as pd
import numpy as np
import pickle
from category_encoders import TargetEncoder
from collections import Counter
import numpy as np
df = pd.read_csv("data/public_dataset.csv")

C:\Users\outra\AppData\Local\Temp\ipykernel_9992\2952253272.py:7: DtypeWarning: Columns (16,17,36,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/public_dataset.csv")


In [19]:
labels = df["Label"].to_list()
print(f"The ratio of labels is: {sum(labels)/len(labels)}")
print(f"The length of the dataset is: {len(labels)}")
print(f"The number of srcip is: {len(set(df['srcip'].to_list()))}")
print(f"The number of dstip is: {len(set(df['dstip'].to_list()))}")

The ratio of labels is: 0.12648702957071267
The length of the dataset is: 2540047
The number of srcip is: 43
The number of dstip is: 47


In [20]:
print(f"The time span is: {df['Stime'].to_list()[-1]-df['Stime'].to_list()[0]}")
start = df['Stime'].to_list()[0]

The time span is: 2334654


In [21]:
print(set(df[df['Label'] == 1]["attack_cat"].to_list()))

{'Reconnaissance', 'Generic', 'Worms', 'Backdoors', 'Analysis', 'Exploits', ' Fuzzers ', ' Reconnaissance ', 'Backdoor', ' Shellcode ', 'Shellcode', ' Fuzzers', 'DoS'}


In [22]:
l = np.nan_to_num(df[df['Label'] == 1]["attack_cat"].to_list())
print(len(l))
print(Counter(l))
print(sum(Counter(l).values()))

321283
Counter({'Generic': 215481, 'Exploits': 44525, ' Fuzzers ': 19195, 'DoS': 16353, ' Reconnaissance ': 12228, ' Fuzzers': 5051, 'Analysis': 2677, 'Backdoor': 1795, 'Reconnaissance': 1759, ' Shellcode ': 1288, 'Backdoors': 534, 'Shellcode': 223, 'Worms': 174})
321283


In [23]:
df["Stime"] = df["Stime"]-start
df["Ltime"] = df["Ltime"]-start

In [24]:
#numbers of cat_columns
print(f"Number of different services: {len(set(df['service'].to_list()))}")
print(f"Number of different proto: {len(set(df['proto'].to_list()))}")
print(f"Number of different state: {len(set(df['state'].to_list()))}")

Number of different services: 13
Number of different proto: 135
Number of different state: 16


In [25]:
print(Counter(df["proto"].tolist()))

Counter({'tcp': 1495074, 'udp': 990435, 'unas': 16202, 'arp': 10064, 'ospf': 7798, 'sctp': 1525, 'icmp': 524, 'any': 411, 'gre': 324, 'rsvp': 274, 'ipv6': 272, 'sun-nd': 262, 'swipe': 262, 'mobile': 262, 'pim': 262, 'sep': 260, 'ipnip': 137, 'ip': 137, 'ggp': 137, 'st2': 137, 'egp': 137, 'cbt': 137, 'emcon': 137, 'nvp': 137, 'igp': 137, 'xnet': 137, 'argus': 137, 'bbn-rcc': 137, 'chaos': 137, 'pup': 137, 'hmp': 137, 'mux': 137, 'dcn': 137, 'prm': 137, 'trunk-1': 137, 'xns-idp': 137, 'trunk-2': 137, 'leaf-1': 137, 'leaf-2': 137, 'irtp': 137, 'rdp': 137, 'iso-tp4': 137, 'netblt': 137, 'mfe-nsp': 137, 'merit-inp': 137, '3pc': 137, 'xtp': 137, 'idpr': 137, 'tp++': 137, 'ddp': 137, 'idpr-cmtp': 137, 'il': 137, 'idrp': 137, 'ipv6-frag': 137, 'sdrp': 137, 'ipv6-route': 137, 'mhrp': 137, 'bna': 137, 'i-nlsp': 137, 'narp': 137, 'ipv6-no': 137, 'tlsp': 137, 'skip': 137, 'ipv6-opts': 137, 'cftp': 137, 'sat-expak': 137, 'kryptolan': 137, 'rvd': 137, 'ippc': 137, 'sat-mon': 137, 'ipcv': 137, 'visa'

In [26]:
print(Counter(df["service"].tolist()))

Counter({'-': 1246397, 'dns': 781668, 'http': 206273, 'ftp-data': 125783, 'smtp': 81645, 'ftp': 49090, 'ssh': 47160, 'pop3': 1533, 'dhcp': 172, 'ssl': 142, 'snmp': 113, 'radius': 40, 'irc': 31})


In [27]:
 #creating instance of one-hot-encoder
cat_columns_targeted = ["proto"]
cat_columns_onehot = ['service','state']

#perform categorical encoding - one hot is too computationally expensive for protocol so I chose Tareted encoding
cat_encoder = TargetEncoder(cols=cat_columns_targeted)
final_df = cat_encoder.fit_transform(df,y=labels)
final_df = pd.get_dummies(final_df, columns=cat_columns_onehot)

C:\Users\outra\PycharmProjects\jodie\venv\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\outra\PycharmProjects\jodie\venv\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [28]:
for column in final_df.columns[4:]:
    correct = pd.to_numeric(final_df[column], errors='coerce').notnull()
    print(f"The {column} is convertible to numeric in ratio: {sum(correct)/len(correct)}")

The dbytes is convertible to numeric in ratio: 1.0
The ct_srv_dst is convertible to numeric in ratio: 1.0
The tcprtt is convertible to numeric in ratio: 1.0
The dtcpb is convertible to numeric in ratio: 1.0
The swin is convertible to numeric in ratio: 1.0
The smeansz is convertible to numeric in ratio: 1.0
The Djit is convertible to numeric in ratio: 1.0
The ackdat is convertible to numeric in ratio: 1.0
The dur is convertible to numeric in ratio: 1.0
The sttl is convertible to numeric in ratio: 1.0
The proto is convertible to numeric in ratio: 1.0
The attack_cat is convertible to numeric in ratio: 0.0
The dsport is convertible to numeric in ratio: 0.9998803171752334
The Dpkts is convertible to numeric in ratio: 1.0
The ct_dst_sport_ltm is convertible to numeric in ratio: 1.0
The trans_depth is convertible to numeric in ratio: 1.0
The dwin is convertible to numeric in ratio: 1.0
The Dload is convertible to numeric in ratio: 1.0
The Sjit is convertible to numeric in ratio: 1.0
The ct_st

In [29]:
# Based on number above we drop some columns and get rid of dsport and sport invalid entries
final_df.drop(columns=["ct_flw_http_mthd", "ct_ftp_cmd", "is_ftp_login"],inplace= True)
final_df = final_df[pd.to_numeric(final_df["dsport"], errors='coerce').notnull()]
final_df = final_df[pd.to_numeric(final_df["sport"], errors='coerce').notnull()]

In [30]:
print(len(final_df))
final_df.rename({"srcip" :"user_id", "dstip" :"item_id", "Stime" :"timestamp", "Label":"state_label"});

2539739


In [31]:
#final_df.to_csv("data/public_dataset_ready.csv",index = False)

In [33]:
percent_of_interactions = 0.4
final_df_reduced = final_df[[True]*int(len(final_df)*percent_of_interactions)+[False]*(len(final_df)-int(len(final_df)*percent_of_interactions))]
attack_cats = np.nan_to_num(final_df_reduced["attack_cat"].to_list())
final_df_reduced.drop(columns=["attack_cat" ], inplace=True)
final_df_reduced.to_csv(f"data/public_dataset_ready_{percent_of_interactions}p.csv",index = False)

C:\Users\outra\AppData\Local\Temp\ipykernel_9992\1290529228.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_reduced.drop(columns=["attack_cat" ], inplace=True)


In [34]:
print(Counter(attack_cats))

Counter({'nan': 993680, 'Generic': 7522, 'Exploits': 5409, ' Fuzzers': 5051, 'Reconnaissance': 1759, 'DoS': 1167, 'Backdoors': 534, 'Analysis': 526, 'Shellcode': 223, 'Worms': 24})


In [35]:
with open("data/reduced_dataset_attack_labels", "wb") as fp:   #Pickling
    pickle.dump(attack_cats, fp)